<a href="https://colab.research.google.com/github/aliabdari/compressed-QA/blob/main/QA_compressed_domain_FE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import glob
import numpy as np
import cv2
from tensorflow.keras.models import Model
import tensorflow.keras.applications.vgg16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
model = VGG16(weights='imagenet', include_top=True)
new_model = Model(model.inputs, model.layers[-2].output)

In [ ]:
def process_frames(frames):
    
    # resized = cv2.resize(frame, (224,224), interpolation = cv2.INTER_AREA)

    # img_data = np.expand_dims(resized, axis=0)
    # img_data = preprocess_input(frames)
    features = new_model.predict(frames)

    return features

In [ ]:
def process_vid(vid):
    num_frame=0
    cap= cv2.VideoCapture(vid)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # feature_vector=np.zeros([2048,length])
    frames = np.zeros([length,224,224,3])
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
             frames[num_frame] = preprocess_input(cv2.resize(frame, (224,224), interpolation = cv2.INTER_AREA))
             num_frame += 1
             if cv2.waitKey(30) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    print("frames = " + str(frames.shape))
    features_transpose =  np.transpose(process_frames(frames =  frames))
    print("features transpose = " + str(features_transpose.shape))
    return features_transpose

In [ ]:
def claculate_number_of_videos(path, extension):
    num_vid = 0
    for vid in glob.glob(path +'/*.' + extension):
        num_vid+=1
    return num_vid

In [ ]:
classes_path = '/content/drive/MyDrive/live_dataset_Reconstructed_Residuals'
numVid = 0
for c in glob.glob(classes_path + '/*'):
  print(c)
  numVid += claculate_number_of_videos(c, 'avi')
print("number of videos which should be processed = " + str(numVid))
idx_vid = 0
idx_class = 0
x_train = np.empty(numVid,dtype=object)
y_train = np.empty(numVid,dtype='int64')
global x_train
global y_train
for c in glob.glob(classes_path + '/*'):
  for vid in glob.glob(c + '/*.avi'):
    #load every video of dataset
    print("processing video number " + str(idx_vid +1) +" of " + str(numVid) + " started")
    current_fv = process_vid(vid)
    x_train[idx_vid] = current_fv
    y_train[idx_vid] = np.array([idx_class])
    idx_vid+=1
    print(str(idx_vid) + " of " + str(numVid) + " video is processed")
  idx_class += 1
print("producing dataset finished")

In [ ]:
np.save('/content/drive/MyDrive/information_live/x_vgg16',x_train)